In [213]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [214]:
df=pd.read_csv('NASDAQ_AAPL.txt')

In [215]:
df.columns=['acao','ds','open','high','low','close','lose']

In [216]:
df['ds']=pd.to_datetime(df['ds'],format='%Y%m%d%H%M')
df['ds']=df['ds'].dt.strftime('%d/%m/%Y')
#df=df.set_index(df['ds'])
df=df.drop(['acao'],axis=1)
df

,ds,open,high,low,close,lose
0,11/10/2010,295.01,295.05,294.82,294.82,5235
1,11/10/2010,294.81,294.90,294.80,294.85,7441
2,11/10/2010,294.85,294.98,294.84,294.85,4268
3,11/10/2010,294.83,294.83,294.75,294.83,4012
4,11/10/2010,294.80,294.82,294.64,294.67,13081
...,...,...,...,...,...,...
2009,08/11/2010,318.59,318.76,318.56,318.76,3561
2010,08/11/2010,318.70,318.70,318.41,318.50,4841
2011,08/11/2010,318.51,318.81,318.51,318.70,5244
2012,08/11/2010,318.81,318.81,318.52,318.60,140566


In [217]:
df=df.sort_values(['ds'])
df=df.groupby(['ds']).mean()
df                 

,open,high,low,close,lose
ds,,,,,
01/11/2010,303.788899,303.992811,303.565939,303.806652,155237.333333
02/11/2010,308.780313,308.997292,308.489271,308.816042,157438.843750
03/11/2010,310.939687,311.201458,310.549583,310.961146,186240.062500
04/11/2010,317.848021,318.095417,317.489896,317.863854,233207.187500
05/11/2010,317.928842,318.130526,317.639158,317.925684,125859.705263
08/11/2010,318.737263,318.906421,318.389684,318.748632,97603.442105
11/10/2010,296.149479,296.322500,295.980938,296.156250,156928.364583
12/10/2010,297.005000,297.250104,296.795000,297.037708,202958.541667
13/10/2010,300.821979,301.013125,300.638750,300.822604,223215.687500


In [218]:
df['t+1']=df['high']

In [219]:
df['t']=df['high'].shift(1)
df=df.dropna()
from sklearn.preprocessing import MinMaxScaler
sc=MinMaxScaler()
df['t']=sc.fit_transform(df.filter(['t']))

/home/naubergois/miniconda3/envs/nauber/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [220]:
x=[]
y=[]

for i in range(len(df)-10):
    x.append(df.iloc[i:i+10]['t'])
    y.append(df.iloc[i+10]['t'])
    



In [221]:
y[1]

0.8935145696344264

In [222]:
x=np.array(x)
x=x.reshape(x.shape[0],x.shape[1],1)

In [223]:
model=tf.keras.models.Sequential()

In [224]:
#model.add(tf.keras.layers.Embedding(input_dim=20000,output_dim=128,input_shape=(10,)))
model.add(tf.keras.layers.LSTM(units=10,activation='relu',return_sequences=True,input_shape=(10,1)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(units=10,activation='relu',return_sequences=True))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(units=20,activation='relu'))
model.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_44 (LSTM)               (None, 10, 10)            480       
_________________________________________________________________
dropout_24 (Dropout)         (None, 10, 10)            0         
_________________________________________________________________
lstm_45 (LSTM)               (None, 10, 10)            840       
_________________________________________________________________
dropout_25 (Dropout)         (None, 10, 10)            0         
_________________________________________________________________
lstm_46 (LSTM)               (None, 20)                2480      
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 21        
Total params: 3,821
Trainable params: 3,821
Non-trainable params: 0
___________________________________________________

In [225]:
model.compile(optimizer='adam',loss='mean_squared_error')

In [236]:
import altair as alt

from sklearn.model_selection import TimeSeriesSplit
ts=TimeSeriesSplit(n_splits=2)
for train_index,test_index in ts.split(x):
    #print(train_index)
    x_=np.array([np.array(x[i]) for i in train_index])
    x_t=np.array([np.array(x[i]) for i in test_index])
    y_=[y[i] for i in train_index]
    y_t=[y[i] for i in test_index]
    print(len(x_))
    model.fit(x_,y_,epochs=300,verbose=0)
    y_pred=model.predict(x_)
    count=0
    dfa=pd.DataFrame()
    for yaux in y_pred:
        dfa=dfa.append({'x':count,'y':pd.to_numeric(yaux[0]),'class':'predicted'},ignore_index=True)
        count+=1
    count=0
    for yaux in y_:
        dfa=dfa.append({'x':count,'y':pd.to_numeric(yaux),'class':'real'},ignore_index=True)
        count+=1
    print(dfa)
        
    base=alt.Chart(dfa).mark_line().encode(
    x='x',
    y='y',
    color='class')
    
    display(base)
    
    



4
       class    x         y
0  predicted  0.0  0.605059
1  predicted  1.0  0.820619
2  predicted  2.0  0.677802
3  predicted  3.0  0.618940
4       real  0.0  0.608822
5       real  1.0  0.893515
6       real  2.0  0.598015
7       real  3.0  0.676809


alt.Chart(...)

7
        class    x         y
0   predicted  0.0  0.624995
1   predicted  1.0  0.931980
2   predicted  2.0  0.663724
3   predicted  3.0  0.628014
4   predicted  4.0  0.611500
5   predicted  5.0  0.597232
6   predicted  6.0  0.590885
7        real  0.0  0.608822
8        real  1.0  0.893515
9        real  2.0  0.598015
10       real  3.0  0.676809
11       real  4.0  0.643457
12       real  5.0  0.539133
13       real  6.0  0.609901


alt.Chart(...)